## Creación de un simple ChatBot que interactúa con llama 3.2

### 1. Librerías

In [1]:
%pip install langchain==0.3.9 langchain-core==0.3.21 langchain-ollama==0.2.0

INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------------------------ --------- 9.7/12.6 MB 46.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 39.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 39.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## 2. Large language model

Hay diferentes modelos que se pueden usar. La mayoría necesita un "API KEY":

ChatOpenAI, ChatAnthropic, ChatVertexAI, ChatCohere, ChatNVIDIA, ChatGroq, ChatMistralAI

Existen otros modelos en la plataforma HUGGINGFACE.

Para la mayoría de los modelos hay que definir una variable de entorno:

------------------------------------------------------------------------------
import getpass

import os

export OPENAI_API_KEY="..."

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

------------------------------------------------------------------------------

Usando Ollama en local no es necesario definir variables de entorno. 


In [3]:
local_llm = 'llama3'
llm = ChatOllama(model=local_llm, temperature=0)

Algunos de los parámetros más importantes que se deben/pueden especificar:

-**model**: el nombre del modelo específico que se quiere usar (por ejemplo, para ChatOpenAI puede ser "gpt-3.5-turbo" o "gpt-4")

**temperature**: controla la aleatoriedad de la respuesta (y la capacidad "generativa"), el valor mínimo es 0 (muy baja aleatoriedad y creatividad).

**timeout**: el máximo tiempo de espera para obtener la respuesta

**max_tokens**: es un número que limita el valor máximo de palabra y puntuación en la respuesta.

**api_key**: el api key del usuario

No todos los modelos admiten los mismos parámetros.

Métodos clave del modelo de chat:

-**invoke**: El método principal para interactuar con un modelo de chat. Acepta una lista de mensajes como entrada y devuelve una lista de mensajes como salida.

-**stream**: Un método que permite transmitir la salida de un modelo de chat a medida que se genera.

-**batch**: Un método que permite agrupar varias solicitudes a un modelo de chat para su procesamiento eficiente.

-**bind_tools**: Un método que permite vincular una herramienta a un modelo de chat para su uso en el contexto de ejecución del modelo.

-**with_structured_output**: Un envoltorio alrededor del método invoke para modelos que admiten natively salida estructurada.


In [4]:
messages= 'Hola, traduce al inglés: hola'
llm.invoke(messages)

AIMessage(content='¡Hola! translates to "Hello!" in English.', additional_kwargs={}, response_metadata={'model': 'llama3', 'created_at': '2025-01-15T22:28:53.997854Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21372349500, 'load_duration': 12156594400, 'prompt_eval_count': 20, 'prompt_eval_duration': 3792000000, 'eval_count': 12, 'eval_duration': 5410000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-218cdf09-f535-4f53-88b9-e44a9b17f36d-0', usage_metadata={'input_tokens': 20, 'output_tokens': 12, 'total_tokens': 32})

### 3. Mensajes

Un mensaje suele consistir en las siguientes piezas de información:

-**Rol**: El rol del mensaje (por ejemplo, "usuario", "asistente").

-**Contenido**: El contenido del mensaje (por ejemplo, texto, datos multimodales).

-**Metadatos adicionales**: id, nombre, uso de tokens y otros metadatos específicos del modelo.

Rol
Los roles se utilizan para distinguir entre diferentes tipos de mensajes en una conversación y ayudar al modelo de chat a entender cómo responder a una secuencia determinada de mensajes.







In [5]:
messages = [
    SystemMessage(content="Translate the following from english to spanish"),
    HumanMessage(content="Hi"),
]


Se puede usar un "parser" para escribir la respuesta sin metedatos:

In [6]:
parser = StrOutputParser()
result = llm.invoke(messages)
parser.invoke(result)

'Hola'

Se pueden juntar el modelo y el parser en una "chain":

In [7]:
chain = llm | parser
chain.invoke(messages)

'Hola'

**Temperature**

Repetimos 5 veces la pregunta "Escribe un color cualquiera" y analizamos las respuestas para diferentes valores del parámetro "temperature". Para un valor de temperature=0:


In [8]:
messages = 'Hola, escribe un color cualquiera'
llm = ChatOllama(model=local_llm, temperature=0)
chain = llm | parser
for i in range(5):
    print(chain.invoke(messages))


¡Hola!

El color que te escribo es... **Azul**
¡Hola!

El color que te escribo es... **Azul**
¡Hola!

El color que te escribo es... **Azul**
¡Hola!

El color que te escribo es... **Azul**
¡Hola!

El color que te escribo es... **Azul**


Para el valor temperature=1:

In [9]:
messages = 'Hola, escribe un color cualquiera"
llm = ChatOllama(model=local_llm, temperature=1)
chain = llm | parser
for i in range(5):
    print(chain.invoke(messages))

SyntaxError: unterminated string literal (detected at line 1) (2759199238.py, line 1)

**top-k y top-P**

Al igual que la temperatura, los parámetros top-K y top-P también se utilizan para controlar la diversidad de la salida del modelo.

Top-K es un número entero positivo que define la cantidad de tokens más probables de los cuales seleccionar el token de salida. Un top-K de 1 selecciona un solo token.

Top-P define el umbral de probabilidad que, una vez excedido de forma acumulativa, los tokens dejan de ser seleccionados como candidatos. Un top-P de 0 es equivalente típicamente al top K con k=1, y un top-P de 1 selecciona típicamente todos los tokens en el vocabulario del modelo.

Ejecute este ejemplo varias veces, cambie la configuración y observe el cambio en la salida.
 
- top k =64  top_p = 0.95

- top k=1 top_p =0

In [10]:
messages = "You are a creative writer. Write a short story about a cat who goes on an adventure"
llm = ChatOllama(model=local_llm, temperature=1, top_k=64,
        top_p=0.95)
chain = llm | parser
print(chain.invoke(messages))

Whiskers twitching, ears perked up with excitement, Luna the curious cat slipped out of the cozy house she called home. She had been cooped up indoors for days, watching the same old birds flit about in the backyard, and her insides were buzzing to explore.

As night began to fall, casting a warm orange glow over the neighborhood, Luna set off on her adventure. Her tail, like a little metronome, kept perfect time as she padded along the sidewalk. The scent of damp earth and blooming flowers filled her nostrils, and her whiskers quivered with anticipation.

Her first stop was the old oak tree in the park. Luna had heard rumors of a hidden tunnel or passageway within its ancient branches. She climbed up the rough bark, using her sharp claws to find holds, until she reached the crook where two great limbs met. There, nestled among the leafy canopy, she discovered a tiny opening just big enough for a curious cat.

Luna slipped through the gap and found herself in a narrow tunnel, winding d

In [31]:
messages = "You are a creative writer. Write a short story about a cat who goes on an adventure"
llm = ChatOllama(model=local_llm, temperature=1, top_k=1,
        top_p=0)
chain = llm | parser
print(chain.invoke(messages))

**The Whiskered Wanderer**

In the sleepy town of Willowdale, where sunbeams danced through the windows and dust motes waltzed in the air, a sleek black cat named Midnight prowled the streets. Her eyes gleamed like polished onyx as she padded silently from house to house, searching for adventure.

Midnight's owner, an elderly lady named Mrs. Jenkins, had grown fond of her mischievous ways and often left treats and toys scattered about the house. But tonight, Midnight yearned for something more – a taste of freedom, a dash of excitement.

As she slipped out into the night air, the world came alive around her. Fireflies twinkled like tiny lanterns, casting a magical glow over the cobblestone streets. The scent of blooming flowers wafted on the breeze, enticing Midnight to explore further.

She padded through alleys and side streets, weaving past sleeping dogs and snoring cats. Her ears perked up at every sound – a chirping cricket, a hooting owl, or the distant rumble of thunder. With ea

In [32]:
messages = "You are a creative writer. Write a short story about a cat who goes on an adventure"
llm = ChatOllama(model=local_llm, temperature=1, top_k=5,
        top_p=0)
chain = llm | parser
print(chain.invoke(messages))

**The Whiskered Wanderer**

In the sleepy town of Willowdale, where sunbeams danced through the windows and dust motes waltzed in the air, a sleek black cat named Midnight prowled the streets. Her eyes gleamed like polished onyx as she padded silently from house to house, searching for adventure.

Midnight's owner, an elderly lady named Mrs. Jenkins, had grown fond of her mischievous ways and often left treats and toys scattered about the house. But tonight, Midnight yearned for something more – a taste of freedom, a dash of excitement.

As she slipped out into the night air, the world came alive around her. Fireflies twinkled like tiny lanterns, casting a magical glow over the cobblestone streets. The scent of blooming flowers wafted on the breeze, enticing Midnight to explore further.

She padded through alleys and side streets, weaving past sleeping dogs and snoring cats. Her ears perked up at every sound – a chirping cricket, a hooting owl, or the distant rumble of thunder. With ea

## 2. Prompt templates

Si se quieren hacer preguntas siempre del mismo tipo, dando las mismas instrucciones al sustema, se pueden definir "prompt templates", que pueden contener variables, definidas entre {}.

In [18]:
system_template = "Translate the following into {language}, write only the translated word:"

In [35]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [36]:
llm = ChatOllama(model=local_llm, temperature=0)
result = prompt_template.invoke({"language": "italian", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian, write only the translated word:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [37]:
chain = prompt_template | llm | parser

In [38]:
chain.invoke({"language": "italian", "text": "hi"})



'Ciao'

In [40]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

chain = prompt_template | llm | parser
chain.invoke({"topic": "cats"})

'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist!'